In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import HeteroData

#### Set random seed

In [2]:
seed = 10
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

#### Load graph structure

In [3]:
# Load graph_structure data: node features, edge_features and edge_index
graph_structure = pd.read_csv('graph_structure/graph_structure.csv', 
                              index_col=0, 
                              header=0).astype(float).fillna(value=0)
branch_index = pd.read_csv('graph_structure/branch_index.csv', 
                           index_col=0, 
                           header=0).to_numpy()
trafo_index = pd.read_csv('graph_structure/trafo_index.csv', 
                          index_col=0, 
                          header=0).to_numpy()
branch_attr_normalized = pd.read_csv('graph_structure/branch_attr_normalized.csv', 
                                     index_col=0, 
                                     header=0).to_numpy()
trafo_attr_normalized = pd.read_csv('graph_structure/trafo_attr_normalized.csv', 
                                    index_col=0, 
                                    header=0).to_numpy()

## Convert edge index to torch tensors
branch_index = torch.tensor(branch_index.T, dtype=torch.long)
trafo_index = torch.tensor(trafo_index.T, dtype=torch.long)

#### Load normalized demand and generation data

In [4]:
# Read load data
load_test_normalized = pd.read_csv('normalized_data/load_test_normalized.csv', 
                                    index_col=0, header=0).to_numpy()

# Load generation data
gen_test_normalized = pd.read_csv('normalized_data/gen_test_normalized.csv', 
                                    index_col=0, header=0).to_numpy()

# Load max generation data
max_gen_test_normalized = pd.read_csv('normalized_data/max_gen_test_normalized.csv', 
                                        index_col=0, header=0).to_numpy()

load_test_normalized.shape, gen_test_normalized.shape, max_gen_test_normalized.shape

((1388, 300), (369, 300), (369, 300))

#### Read gen and load buses

In [5]:
# Load wind bus bidx
wind_bus = pd.read_csv('graph_structure/_wind_bus.csv', 
                       index_col=None, 
                       header=None).to_numpy().flatten()
wind_bus_bidx = pd.read_csv('graph_structure/_wind_bus_bidx.csv', 
                            index_col=None, 
                            header=None).to_numpy().astype(bool).flatten()
wind_bus_in_all_bidx = pd.read_csv('graph_structure/_wind_bus_in_all_bidx.csv', 
                                   index_col=None, 
                                   header=None).to_numpy().astype(bool).flatten()

# Load gen and load bus bidx
gen_bus_bidx = pd.read_csv('graph_structure/_gen_bus_bidx.csv', 
                           index_col=None, 
                           header=None).to_numpy().astype(bool).flatten()
load_bus_bidx = pd.read_csv('graph_structure/_load_bus_bidx.csv', 
                            index_col=None, 
                            header=None).to_numpy().astype(bool).flatten()

wind_bus.shape, wind_bus_bidx.shape, wind_bus_in_all_bidx.shape, gen_bus_bidx.shape, load_bus_bidx.shape

((152,), (369,), (2848,), (2848,), (2848,))

#### Generate testing data

In [6]:
## Load thermal generator bidx
thermal_bus_bidx = pd.read_csv('graph_structure/_thermal_bus_bidx.csv',
                                index_col=None,
                                header=None).to_numpy().astype(bool).flatten()

## Create traing mask
mask = torch.tensor(thermal_bus_bidx, dtype=torch.bool)

In [7]:
# Store multiple Data() in a list
data_list = []

test_size = load_test_normalized.shape[1]

for i in range(test_size):
    graph_structure['load_p_mw'][load_bus_bidx] = load_test_normalized[:, i]
    graph_structure['max_gen_p_mw'][gen_bus_bidx] = max_gen_test_normalized[:, i]
    graph_structure['min_gen_p_mw'][wind_bus_in_all_bidx] = max_gen_test_normalized[wind_bus_bidx, i]

    X = graph_structure.to_numpy()
    X = torch.from_numpy(X).float()
    
    y = gen_test_normalized[~wind_bus_bidx, i]
    y = torch.from_numpy(y.flatten()).float()

    data = HeteroData()
    data['node'].x = X
    data['node'].y = y
    data.mask = mask

    # Set edge index
    data['node', 'branch', 'node'].edge_index = branch_index
    data['node', 'trafo', 'node'].edge_index = trafo_index
    data['node', 'branch', 'node'].edge_attr = torch.from_numpy(branch_attr_normalized).float()
    data['node', 'trafo', 'node'].edge_attr = torch.from_numpy(trafo_attr_normalized).float()
    
    data_list.append(data)

#### Generate train dataset

In [8]:
dir = f'test_dataset'
if not os.path.exists(dir):
    print(f'There is no such firectory!')
else:
    shutil.rmtree(dir)
    print(f'The old dataset has been deleted!')

The old dataset has been deleted!


In [9]:
from CustomDataset import CustomDataset
CustomDataset(root=dir, data_list=data_list)

Processing...
Done!


CustomDataset(300)